# Training an agent with Regression Network

So we have generated our training dataset. Now we are going to use this notebook to traing a regular logistic regression model. We have used dataiku tool extensivelly so mosy of the code here are copied from dataiku generated notebook.

In [1]:
# Execute the following cell only ones. If you need to re-execute it. First restart the notebook. It is needed to access the data folder.
%cd ..

/Users/hakanonal/vscode/hang-man-ai


In [2]:
import pandas as pd

ml_dataset = pd.read_csv('data/states.csv',usecols=['action_to_play', 'length', 'guessed_letters', 'known_count', 'board', 'unknown_count'])
ml_dataset.head(5)

,guessed_letters,board,length,known_count,unknown_count,action_to_play
0,NaN,"_,_",2,0,0,a
1,a,"a,_",2,1,0,z
2,"a,z","a,_",2,1,1,r
3,"a,z,r","a,_",2,1,2,l
4,"a,z,r,l","a,_",2,1,3,t


In [3]:
def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = ['guessed_letters', 'board']
numerical_features = ['length', 'known_count', 'unknown_count']
text_features = []
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    ml_dataset[feature] = ml_dataset[feature].astype('double')

In [4]:
target_map = {'a': 0, 'l': 1, 'ı': 2, 'k': 3, 'm': 4, 'r': 5, 'n': 6, 'e': 7, 'i': 8, 'd': 9, 't': 10, 'ç': 11, 'c': 12, 'b': 13, 's': 14, 'ş': 15, 'y': 16, 'z': 17, 'f': 18, 'ğ': 19, 'o': 20, 'u': 21, 'v': 22, 'g': 23, 'p': 24, 'h': 25, 'ü': 26, 'ö': 27, 'j': 28}
ml_dataset['__target__'] = ml_dataset['action_to_play'].map(str).map(target_map)
del ml_dataset['action_to_play']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]